In [2]:
using CSVFiles, DataFrames, DataFramesMeta, CategoricalArrays, StatsBase, Distributions, Random, FreqTables, BenchmarkTools

# discourse thread: https://discourse.julialang.org/t/gibbs-sampler-works-in-r-doesnt-in-julia-full-julia-code-provided-please-help/16544/17    

In [182]:
d = DataFrame(load("https://web.stanford.edu/~hastie/CASI_files/DATA/haplotype.csv"))

,,race,Snp1,Snp2,Snp3,Snp4,Snp5,Snp6,Snp7,Snp8,Snp9,Snp10,Snp11,Snp12,Snp13,Snp14,Snp15,Snp16,Snp17,Snp18,Snp19,Snp20,Snp21,Snp22,Snp23,Snp24,Snp25,Snp26,Snp27,Snp28,Snp29,Snp30,Snp31,Snp32,Snp33,Snp34,Snp35,Snp36,Snp37,Snp38,Snp39,Snp40,Snp41,Snp42,Snp43,Snp44,Snp45,Snp46,Snp47,Snp48,Snp49,Snp50,Snp51,Snp52,Snp53,Snp54,Snp55,Snp56,Snp57,Snp58,Snp59,Snp60,Snp61,Snp62,Snp63,Snp64,Snp65,Snp66,Snp67,Snp68,Snp69,Snp70,Snp71,Snp72,Snp73,Snp74,Snp75,Snp76,Snp77,Snp78,Snp79,Snp80,Snp81,Snp82,Snp83,Snp84,Snp85,Snp86,Snp87,Snp88,Snp89,Snp90,Snp91,Snp92,Snp93,Snp94,Snp95,Snp96,Snp97,Snp98,Snp99,Snp100
,String,String,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64⍰,Int64⍰,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64⍰,Int64,Int64⍰,Int64⍰,Int64,Int64⍰,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64⍰,Int64,Int64,Int64⍰,Int64,Int64,Int64,Int64⍰,Int64⍰,Int64,Int64,Int64,Int64⍰,Int64⍰,Int64
1,NA19919,African American,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,2,0,1,1,0,1,0,0,2,0,0,0,1,0,0,1,1,0,0,1,1,1,2,0,0,0,0,2,0,1,0,0,0,2,0,0,1,0,0,0,1,1,2,1,1,0,1,0,0,1,0,2,0,1,0,0,0,1,2,1,1,1,0,0,1,1,1,1,0,2,1,1,0,1,0,0,0,0,0,0,1
2,NA19916,African American,0,0,0,0,0,1,0,0,0,2,0,1,2,0,0,1,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,0,0,0,1,1,0,0,1,1,1,1,0,1,0,1,0,2,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0
3,NA19835,African American,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,1,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,1,1,1,2,0,0,0,2,1,1,0,2,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,1
4,NA20282,African American,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,2,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,2,1,1,1,1,1,1,2,0,1,0,2,0,0,0,0,1,0,0,0,0,0,1,2,2,0,1,0,1,1,0,0,0,1,0,0,0,1,1,0,2
5,NA19703,African American,1,0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,2,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,2,0,0,0,2,2,2,0,2,2,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,2,2,0,0,0,1,1,1,1,1,0,0,1,0,1,2
6,NA19902,African American,0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,2,0,1,0,0,1,0,0,0,0,0,0,0,0,2,0,1,0,0,1,1,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,1,0,2,1,0,0,0,0,0,2,0,1,0,1,1,0,0,1,1,1,2,2,2,1,0,0,0,0,1,2,1,0,1,0,0,0,1
7,NA19901,African American,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,1,0,2,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,2,0,0,0,0,1,0,0,1,0,1,1,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,2
8,NA19908,African American,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,2,0,2,0,1,2,0,1,0,0,1,0,0,0,2,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,2,0,1,0,2,1,1,1,1,1,2,1,0,1,0,1,0,2,0,0,0,1,1,1,2,2,1,0,0,2,0,0,0,2,0,1,0,1,0,0,0,0,0,0,2
9,NA19914,African American,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,1,1,1,1,0,2,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,1,1,1,2,2,0,1,0,1,0,1,1,0,0,1,0,0,1,0


In [213]:
## describe the data frame
ds = describe(d)

## see the `nmissing` column.. it's a union of missing data.
## if you want to change the type of the column to `Array{Int64}` only, use 
## rand(Binomial(1, 0.5))

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,DataType
1,subject,,NA06984,,NA20290,197,,String
2,race,,African,,Japanese,4,,String
3,Snp1,0.15736,0,0.0,2,,0,Int64
4,Snp2,0.152284,0,0.0,1,,,Int64
5,Snp3,0.00507614,0,0.0,1,,,Int64
6,Snp4,0.309645,0,0.0,2,,,Int64
7,Snp5,0.0101523,0,0.0,1,,,Int64
8,Snp6,0.507614,0,0.0,2,,,Int64
9,Snp7,0.329949,0,0.0,2,,,Int64


In [191]:
## rename the columns
rename!(d, names(d)[1] => :subject)

In [137]:
# TO DO: implement https://stackoverflow.com/questions/32276887/use-of-lapply-sd-in-data-table-r

In [199]:
## count how many missing data            
missingl = [(r, c) for r = 1:size(d, 1), c = 1:size(d, 2) if ismissing(d[r, c])]
println("missing: $(length(missingl))")
@show missingl
## get all the ones. 
ones = [(r, c) for r = 1:size(d, 1), c = 1:size(d, 2) if coalesce(d[r, c], 0) == 1]        
# for now set the missing data to zeros
for (r, c) in missingl                        
   d[r, c] = 0
end
                        
@assert length([(r, c) for r = 1:size(d, 1), c = 1:size(d, 2) if ismissing(d[r, c])]) == 0
ones
                                    

missing: 0
missingl = Tuple{Int64,Int64}[]


5597-element Array{Tuple{Int64,Int64},1}:
 (5, 3)    
 (7, 3)    
 (18, 3)   
 (22, 3)   
 (45, 3)   
 (48, 3)   
 (51, 3)   
 (53, 3)   
 (62, 3)   
 (63, 3)   
 (73, 3)   
 (77, 3)   
 (80, 3)   
 ⋮         
 (175, 102)
 (180, 102)
 (183, 102)
 (185, 102)
 (187, 102)
 (188, 102)
 (189, 102)
 (191, 102)
 (192, 102)
 (193, 102)
 (194, 102)
 (197, 102)

In [211]:
## random matrices (Arrays) of 1/0
X1_ones = [rand(Binomial(1, 0.5)) for i = 1:length(ones)*2]
X2_ones = 1 .- X1_ones
@show X1_ones[1]
@show X2_ones[1]

X1_ones[1] = 0
X2_ones[1] = 1


1

In [262]:
## get the columns that are data only
datacolumns = names(d)[3:102];
data_snp = d[datacolumns]
#X = colwise(x -> sum(x), data_snp)
## alternatively X = map(x -> sum(x), DataFrames.columns(data_snp))
bin = Binomial()
local where_one, where_x, x1

mytestdata = [0, 0, 1, 1, 1, 2, 2]

where_x = findall(y -> y ==1, mytestdata)
where_two = findall(y -> y == 2, mytestdata)
x1 = copy(mytestdata)
x2 = copy(mytestdata)

Random.seed!(2342)
x1[where_x] = rand(bin, length(where_x))
x1
where_x1 = findall(y -> y == 1, x1)
x2[where_x1] = fill(0, length(where_x1))
x2
other_ones = setdiff(where_x, where_x1)
x2[other_ones] = fill(1, length(other_ones))
x2

x1[where_two] = fill(1, length(where_two))
x2[where_two] = fill(1, length(where_two))

x1


7-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1
 1